In [1]:
from part_3_config import config as cfg
import preprocess as pp
import numpy as np
import wandb
import torch
import os

In [2]:
TRAIN_LEN = cfg['parameters']['train_len']['value']
VALIDATION_LEN = cfg['parameters']['validation_len']['value']
TEST_LEN = cfg['parameters']['test_len']['value']

# TRAIN_LEN = 1000
# VALIDATION_LEN = 100
# TEST_LEN = 100

MAX_LEN = cfg['parameters']['max_len']['value']

dir = 'RES/TF/TEST'

In [3]:
train_sents, validation_sents, test_sents = pp.get_sents('Auguste_Maquet.txt', TRAIN_LEN, VALIDATION_LEN, TEST_LEN)

5351663
30000
10000
12590
oh, i don t say that your excellency will do as you please i should be sorry to advise you in the matter.


In [4]:
import os

if not os.path.exists(dir):
    os.makedirs(dir)

In [5]:
from torch.utils.data import Dataset
from gensim.models import KeyedVectors
import torch

class SentencesDataset(Dataset):
    def __init__(self, sentences: list, Emb: KeyedVectors, max_len: int = None):
        super().__init__()

        if max_len is not None:
            SentencesDataset.max_len = max_len + 1

        self.X = []
        self.Y = []

        for sentence in sentences:
            s = pp.get_sentence_index(sentence, Emb)
            max_sentence_len = min(SentencesDataset.max_len, len(s))

            self.X.append(torch.cat((s[:max_sentence_len], torch.empty(SentencesDataset.max_len - max_sentence_len, dtype=torch.long).fill_(Emb.key_to_index['pad']))))

            # self.Y.append(s[max_sentence_len])
            # for i in range(max_sentence_len):
            #     self.X.append(torch.cat((s[:i], torch.empty(SentencesDataset.max_len - i, dtype=torch.long).fill_(Emb.key_to_index['pad']))))
            #     self.Y.append(s[i])

        self.X = torch.stack(self.X)
        # self.Y = torch.stack(self.Y)

    def __len__(self):
        return self.X.shape[0]

    def __getitem__(self, idx):
        # return self.X[idx], self.Y[idx]
        return self.X[idx]

In [6]:
from torch.utils.data import DataLoader

def get_dataloader(dataset, batch_size, shuffle):
    return DataLoader(dataset, batch_size=batch_size, shuffle=shuffle, num_workers=4)

In [7]:
def load_data(Emb, batch_size, device, max_len):
    train_dataset = SentencesDataset(train_sents, Emb, max_len)
    validation_dataset = SentencesDataset(validation_sents, Emb)
    test_dataset = SentencesDataset(test_sents, Emb)

    train_dataloader = get_dataloader(train_dataset, batch_size, True)
    validation_dataloader = get_dataloader(validation_dataset, batch_size, True)
    test_dataloader = get_dataloader(test_dataset, batch_size, False)

    return train_dataloader, validation_dataloader, test_dataloader

In [8]:
import tqdm
import numpy as np

def run(model, dataloader, train, es, device, loss_fn, optimizer, epoch):
    if train:
        model.train()
    else:
        model.eval()

    epoch_loss = []

    pbar = tqdm.tqdm(dataloader)

    for X in pbar:
        # print(X[0])
        Y = X[:, 1:]
        X = X[:, :-1]

        # print(X[0])
        # print(Y[0])

        # break

        Y_pred = model(X)
        Y = Y.to(device)

        Y_pred = Y_pred.view(-1, Y_pred.shape[-1])
        Y = Y.view(-1)

        loss = loss_fn(Y_pred, Y)
        epoch_loss.append(loss.item())

        if train:
            optimizer.zero_grad()
            loss.backward()
            optimizer.step()

        pbar.set_description(f'{epoch} {"T" if train else "V"} Loss: {loss.item():7.4f}, Avg Loss: {sum(epoch_loss) / len(epoch_loss):7.4f}, Best Loss: {es.best_loss:7.4f}, Counter: {es.counter}')

    return np.mean(epoch_loss)

In [9]:
import wandb

def train_epoch(model, train_dataloader, validation_dataloader, es, device, loss_fn, optimizer, epoch):
    train_loss = run(model, train_dataloader, True, es, device, loss_fn, optimizer, epoch)
    wandb.log({'train_loss': train_loss})
    with torch.no_grad():
        validation_loss = run(model, validation_dataloader, False, es, device, loss_fn, optimizer, epoch)
        wandb.log({'validation_loss': validation_loss})
    print(f'Epoch {epoch} Train Loss: {train_loss:7.4f}, Validation Loss: {validation_loss:7.4f}')
    return train_loss, validation_loss

In [10]:
from transformer import Decoder
from EarlyStopping import EarlyStopping
import torch.nn as nn

def train(train_dataloader, validation_dataloader, cfg, Emb):

    # nhead = cfg['parameters']['nhead']['value']
    # dim_feedforward = cfg['parameters']['dim_feedforward']['value']
    # num_layers = cfg['parameters']['num_layers']['value']
    # dropout = cfg['parameters']['dropout']['value']
    # max_len = cfg['parameters']['max_len']['value']
    # epochs = cfg['parameters']['epochs']['value']
    # learning_rate = cfg['parameters']['learning_rate']['value']
    # optimizer = cfg['parameters']['optimizer']['value']

    nhead = cfg.nhead
    dim_feedforward = cfg.dim_feedforward
    num_layers = cfg.num_layers
    dropout = cfg.dropout
    max_len = cfg.max_len
    epochs = cfg.epochs
    learning_rate = cfg.learning_rate
    optimizer = cfg.optimizer

    model = Decoder(Emb, nhead, dim_feedforward, num_layers, dropout, max_len, pp.device).to(pp.device)
    # print(model)

    loss_fn = nn.CrossEntropyLoss()
    optimizer = getattr(torch.optim, optimizer)(model.parameters(), lr=learning_rate)
    # optimizer = torch.optim.Adam(model.parameters(), lr=learning_rate)

    es = EarlyStopping(patience=3)

    for epoch in range(epochs):
        _, validation_loss = train_epoch(model, train_dataloader, validation_dataloader, es, pp.device, loss_fn, optimizer, epoch)
        # Save model
        torch.save(model.state_dict(), os.path.join(dir, f'nnlm_{epoch}.pth'))

        if es(validation_loss, epoch):
            break

    os.rename(os.path.join(dir, f'nnlm_{es.best_model_pth}.pth'), os.path.join(dir, f'best_model.pth'))

    return es.best_loss

In [11]:
from nnlm import NNLM
import tqdm

def run_perplexity(dataloader, best_model, best_pth, Emb):
    best_model.load_state_dict(torch.load(best_pth))
    best_model.eval()

    loss_fn = nn.CrossEntropyLoss()

    with torch.no_grad():
        perplexity = []

        pbar = tqdm.tqdm(dataloader)
        for X in pbar:
            Y = X[:, 1:]
            X = X[:, :-1]

            # tgt_key_padding_mask = (X == Emb.key_to_index['pad']).transpose(0, 1)

            Y_pred = best_model(X)
            Y = Y.to(pp.device)

            for i in range(Y_pred.shape[0]):
                pval = 0
                pix = 0
                sentence = ''
                
                # print(X.shape)
                # print(Y_pred.shape)
                # print(Y.shape)

                for j in range(1, X.shape[1]):
                    if X[i][j].item() == Emb.key_to_index['eos']:
                        break
                    sentence += Emb.index_to_key[X[i][j].item()] + ' '
                    pix = j + 1

                # print(sentence.strip())
                # print(Y_pred[i][:pix].shape)
                # print(Y[i][:pix].shape)
                # print(Y_pred[i][:pix])
                # print(Y[i][:pix])

                # print(loss_fn(Y_pred[i][:pix], Y[i][:pix]).item())

                pval = np.exp(loss_fn(Y_pred[i][:pix], Y[i][:pix]).item())
                # print(f'{sentence.strip()}: {pval}', file=f)
                perplexity.append(pval)

            #     break
            # break

        # print(f'Perplexity: {np.mean(perplexity)}')
        return np.mean(perplexity)

def get_all_perplexity_vals(test_dataloader, cfg, Emb):

    # nhead = cfg['parameters']['nhead']['value']
    # dim_feedforward = cfg['parameters']['dim_feedforward']['value']
    # num_layers = cfg['parameters']['num_layers']['value']
    # dropout = cfg['parameters']['dropout']['value']
    # max_len = cfg['parameters']['max_len']['value']

    nhead = cfg.nhead
    dim_feedforward = cfg.dim_feedforward
    num_layers = cfg.num_layers
    dropout = cfg.dropout
    max_len = cfg.max_len

    best_model = Decoder(Emb, nhead, dim_feedforward, num_layers, dropout, max_len, pp.device).to(pp.device)
    # print(best_model)
    best_pth = os.path.join(dir, 'best_model.pth')

    # with open(os.path.join(dir, 'test_perplexity.txt'), 'w') as f:
    return run_perplexity(test_dataloader, best_model, best_pth, Emb)
    
    # return run_perplexity(test_dataloader, best_model, best_pth, Emb)

In [12]:
# WANDB init
import wandb

def run_everything(cfg=None):
    with wandb.init(config=cfg):
        cfg = wandb.config

        # embedding_dim = cfg['parameters']['embedding_dim']['value']
        # batch_size = cfg['parameters']['batch_size']['value']
        # max_len = cfg['parameters']['max_len']['value']

        embedding_dim = cfg.embedding_dim
        batch_size = cfg.batch_size
        max_len = cfg.max_len

        Emb = pp.create_vocab(train_sents, embedding_dim)
        print(len(Emb.key_to_index))

        train_dataloader, validation_dataloader, test_dataloader = load_data(Emb, batch_size, pp.device, max_len)

        val_loss = train(train_dataloader, validation_dataloader, cfg, Emb)
        wandb.log({'best_loss': val_loss})

        train_perplexity = get_all_perplexity_vals(train_dataloader, cfg, Emb)
        test_perplexity = get_all_perplexity_vals(test_dataloader, cfg, Emb)

        wandb.log({'train_perplexity': train_perplexity})
        wandb.log({'test_perplexity': test_perplexity})

sweep_id = wandb.sweep(cfg, project="Transformer")
wandb.agent(sweep_id, run_everything, count=50)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


Create sweep with ID: 6t5kvtay
Sweep URL: https://wandb.ai/shu7bh/Transformer/sweeps/6t5kvtay


wandb: Agent Starting Run: fd41icek with config:
wandb: 	batch_size: 32
wandb: 	dim_feedforward: 512
wandb: 	dropout: 0
wandb: 	embedding_dim: 200
wandb: 	epochs: 100
wandb: 	learning_rate: 0.001
wandb: 	loss: CrossEntropyLoss
wandb: 	max_len: 50
wandb: 	nhead: 4
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	test_len: 14000
wandb: 	train_len: 30000
wandb: 	validation_len: 10000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shu7bh. Use `wandb login --relogin` to force relogin


16244
Decoder(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=200, out_features=200, bias=True)
        )
        (linear1): Linear(in_features=200, out_features=512, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=512, out_features=200, bias=True)
        (norm1): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((200,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p

0 T Loss:  1.8468, Avg Loss:  2.1582, Best Loss:     inf, Counter: 0: 100%|██████████| 938/938 [00:20<00:00, 45.84it/s]
  0%|          | 0/313 [00:00<?, ?it/s]/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
0 V Loss:  2.4069, Avg Loss:  1.9215, Best Loss:     inf, Counter: 0: 100%|██████████| 313/313 [00:03<00:00, 86.45it/s] 


...

wandb: Agent Starting Run: ajz2f6n2 with config:
wandb: 	batch_size: 32
wandb: 	dim_feedforward: 512
wandb: 	dropout: 0
wandb: 	embedding_dim: 100
wandb: 	epochs: 100
wandb: 	learning_rate: 0.005
wandb: 	loss: CrossEntropyLoss
wandb: 	max_len: 50
wandb: 	nhead: 4
wandb: 	num_layers: 2
wandb: 	optimizer: Adam
wandb: 	test_len: 14000
wandb: 	train_len: 30000
wandb: 	validation_len: 10000
Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.


16244


wandb: Ctrl + C detected. Stopping sweep.


Decoder(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=512, bias=True)
        (dropout): Dropout(p=0, inplace=False)
        (linear2): Linear(in_features=512, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0, inplace=False)
        (dropout2): Dropout(p=0, in

0 T Loss:  2.1434, Avg Loss:  3.2927, Best Loss:     inf, Counter: 0:   4%|▎         | 34/938 [00:01<00:21, 41.68it/s]Exception ignored in: <generator object tqdm.__iter__ at 0x7f3d54f12c30>
Traceback (most recent call last):
  File "/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/tqdm/std.py", line 1193, in __iter__
    self.close()
  File "/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/tqdm/std.py", line 1287, in close
    fp_write('')
  File "/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/tqdm/std.py", line 1284, in fp_write
    self.fp.write(str(s))
  File "/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/tqdm/utils.py", line 127, in inner
    return func(*args, **kwargs)
  zmq.error.ZMQError: Socket operation on non-socket
Exception ignored in sys.unraisablehook: <built-in function unraisablehook>
Traceback (most recent call last):
  File "/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/ipykernel/iost

In [ ]:
# run_everything(cfg)

Failed to detect the name of this notebook, you can set it manually with the WANDB_NOTEBOOK_NAME environment variable to enable code saving.
wandb: Currently logged in as: shu7bh. Use `wandb login --relogin` to force relogin


16244
Decoder(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dropout(p=0.1, inplace=False)
        (dropout2): D

0 T Loss:  1.8565, Avg Loss:  2.2712, Best Loss:     inf, Counter: 0: 100%|██████████| 938/938 [00:21<00:00, 42.79it/s]
  0%|          | 0/313 [00:00<?, ?it/s]/home2/shu7bh/miniconda3/envs/main/lib/python3.11/site-packages/torch/nn/modules/activation.py:1160: UserWarning: Converting mask without torch.bool dtype to bool; this will negatively affect performance. Prefer to use a boolean mask directly. (Triggered internally at /opt/conda/conda-bld/pytorch_1682343995622/work/aten/src/ATen/native/transformers/attention.cpp:150.)
  return torch._native_multi_head_attention(
0 V Loss:  2.1339, Avg Loss:  1.9637, Best Loss:     inf, Counter: 0: 100%|██████████| 313/313 [00:03<00:00, 84.72it/s] 


Epoch 0 Train Loss:  2.2712, Validation Loss:  1.9637


1 T Loss:  1.6589, Avg Loss:  1.9537, Best Loss:  1.9637, Counter: 0: 100%|██████████| 938/938 [00:20<00:00, 44.82it/s]
1 V Loss:  2.4867, Avg Loss:  1.8854, Best Loss:  1.9637, Counter: 0: 100%|██████████| 313/313 [00:03<00:00, 87.93it/s] 


...

Epoch 13 Train Loss:  1.6173, Validation Loss:  1.7945


14 T Loss:  1.2294, Avg Loss:  1.6061, Best Loss:  1.7863, Counter: 3: 100%|██████████| 938/938 [00:20<00:00, 45.60it/s]
14 V Loss:  1.6619, Avg Loss:  1.7976, Best Loss:  1.7863, Counter: 3: 100%|██████████| 313/313 [00:03<00:00, 84.71it/s] 


Epoch 14 Train Loss:  1.6061, Validation Loss:  1.7976
Decoder(
  (pos_encoder): PositionalEncoding(
    (dropout): Dropout(p=0.1, inplace=False)
  )
  (transformer_decoder): TransformerDecoder(
    (layers): ModuleList(
      (0-1): 2 x TransformerDecoderLayer(
        (self_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (multihead_attn): MultiheadAttention(
          (out_proj): NonDynamicallyQuantizableLinear(in_features=100, out_features=100, bias=True)
        )
        (linear1): Linear(in_features=100, out_features=1024, bias=True)
        (dropout): Dropout(p=0.1, inplace=False)
        (linear2): Linear(in_features=1024, out_features=100, bias=True)
        (norm1): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm2): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (norm3): LayerNorm((100,), eps=1e-05, elementwise_affine=True)
        (dropout1): Dr

100%|██████████| 394/394 [00:10<00:00, 38.32it/s]


best_loss,▁
test_perplexity,▁
train_loss,█▅▄▃▃▃▂▂▂▂▁▁▁▁▁
validation_loss,█▅▃▂▂▁▁▁▁▁▁▁▁▁▁
best_loss,1.78628
test_perplexity,98.46982
train_loss,1.60607
validation_loss,1.79763


In [ ]:
# from part_3_config import config as cfg
# import preprocess as pp

# embedding_dim = cfg['parameters']['embedding_dim']['value']
# batch_size = cfg['parameters']['batch_size']['value']
# max_len = cfg['parameters']['max_len']['value']

# Emb = pp.create_vocab(train_sents, embedding_dim)
# train_dataloader, validation_dataloader, test_dataloader = load_data(Emb, batch_size, pp.device, max_len)

In [ ]:
# get_all_perplexity_vals(test_dataloader, cfg, Emb)

In [ ]:
# nhead = cfg['parameters']['nhead']['value']
# dim_feedforward = cfg['parameters']['dim_feedforward']['value']
# num_layers = cfg['parameters']['num_layers']['value']
# dropout = cfg['parameters']['dropout']['value']
# max_len = cfg['parameters']['max_len']['value']

# best_model = Decoder(Emb, nhead, dim_feedforward, num_layers, dropout, max_len, pp.device).to(pp.device)
# print(best_model)
# best_pth = os.path.join(dir, 'best_model.pth')

# # generate a sentence from the model

# q = 'my name is '
# print(q, end=' ')

# best_model.load_state_dict(torch.load(best_pth))
# best_model.eval()

# with torch.no_grad():
#     for i in range(10):
#         e = pp.get_sentence_index(q, Emb)
#         e = e[:-1]
#         # print(e)
#         X = torch.cat((e, torch.empty(max_len - len(e), dtype=torch.long).fill_(Emb.key_to_index['pad'])))
#         # print (X)
#         Y_pred = best_model(X)
#         # print(Y_pred.shape)
#         Y_pred = Y_pred[0][len(e) - 1]
#         Y_pred = torch.softmax(Y_pred, dim=-1)
#         # Y_pred = torch.multinomial(Y_pred, num_samples=1)
#         Y_pred = torch.argmax(Y_pred)


#         # print(Y_pred.shape)

#         # for j in range(len(e)):
#         #     a = torch.softmax(Y_pred[0, j], dim=0)
#         #     # b = torch.multinomial(a, num_samples=1)
#         #     b = torch.argmax(a)
#         #     print(Emb.index_to_key[e[j].item()], Emb.index_to_key[b.item()])
#         # print (Y_pred)
#         # Y_pred = Y_pred[0, len(e) - 1]
#         # # Y_pred[len(e)]
#         # Y_pred = torch.softmax(Y_pred, dim=-1)
#         # print(Y_pred)
#         # Y_pred = torch.multinomial(Y_pred, num_samples=1)

#         q += ' ' + Emb.index_to_key[Y_pred.item()]
#         # print the word
#         print(Emb.index_to_key[Y_pred.item()], end=' ')